<a href="https://colab.research.google.com/github/branjbar/nlp-bert-family/blob/master/transformers/finetuning_roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://huggingface.co/transformers/examples.html#roberta-bert-and-masked-language-modeling
# https://www.kaggle.com/jihunlorenzopark/pretrain-with-another-twitter-dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Sat May  2 12:20:38 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================

In [2]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (361/361), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 26096 (delta 178), reused 322 (delta 163), pack-reused 25735
Receiving objects: 100% (26096/26096), 15.38 MiB | 14.35 MiB/s, done.
Resolving deltas: 100% (18222/18222), done.


In [3]:
import os
os.chdir("transformers")
!pip install .

Processing /content/transformers
     |████████████████████████████████| 3.8MB 6.5MB/s 
     |████████████████████████████████| 1.0MB 28.6MB/s 
     |████████████████████████████████| 890kB 44.1MB/s 
  Created wheel for transformers: filename=transformers-2.8.0-cp36-none-any.whl size=595726 sha256=93508cc47e01a6f1a612b9d151ec28de0b9ae8db37ea7ce91519c8c9c2b0e90e
  Stored in directory: /tmp/pip-ephem-wheel-cache-_zskg9s2/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=fb31c0c8c694fc377da5bbbcd56fb3b433c02755c67aed9f198ac5ce239376ec
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built transformers sacremoses


In [0]:
# !wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
# !unzip wikitext-2-raw-v1.zip

In [0]:
# !python examples/run_language_modeling.py \
#     --output_dir=output \
#     --model_type=roberta \
#     --model_name_or_path=roberta-base \
#     --do_train \
#     --train_data_file=wikitext-2-raw/wiki.test.raw \
#     --do_eval \
#     --eval_data_file=wikitext-2-raw/wiki.test.raw \
#     --mlm

In [0]:
from sklearn.model_selection import train_test_split
import pandas as pd, re, numpy as np


In [0]:
KAGGLE_INPUT_PATH = "../input/"
COLAB_INPUT_PATH = "/content/drive/My Drive/input/"
INPUT_PATH = COLAB_INPUT_PATH

DATA_PATH = INPUT_PATH + "tweet-sentiment-extraction/"
CHECKPOINT_PATH = INPUT_PATH + "model_checkpoint/roberta/"
ROBERTA_PATH = INPUT_PATH + 'tf-roberta/'
EMOTION_PATH = INPUT_PATH + 'training-data/emotion/text_emotion.csv'
SENTIMENT140_PATH = INPUT_PATH + 'training-data/sentiment140/training.1600000.processed.noemoticon.csv'
TWITTER_SENTIMENTS_PATH = INPUT_PATH + 'training-data/twitter-sentiment/Sentiment Analysis Dataset 2.csv'

In [0]:
def remove_html(text):
    text = re.sub("&quot;", '"', text)
    text = re.sub("&gt;", ">", text)
    text = re.sub("&lt;", "<", text)
    text = re.sub("&le;", "≤", text)
    text = re.sub("&ge;", "≥", text)
    text = re.sub("&amp;", "&", text)
    return text

def remove_twitter_id(text):
    text = text.strip()
    text = " ".join([i for i in text.split(" ") if "@" not in i])
    text = text.strip()
    return text

def to_text(df, filename):
    f = open(filename, 'w')
    for c in df["content"].values:
        f.write(" " + c.strip() + "\n\n")
    f.close()

def pandas_to_train_test_file(df, filename):
    df_train, df_test = train_test_split(df, test_size=0.1, random_state=333)
    to_text(df_train, "../%s.train.txt"%filename)
    to_text(df_test, "../%s.test.txt"%filename)

In [9]:
emotion_df = pd.read_csv(EMOTION_PATH)
emotion_df['content'] = emotion_df['content'].astype(str).str.lower()
emotion_df["content"] = emotion_df["content"].apply(remove_html)
emotion_df["content"] = emotion_df["content"].apply(remove_twitter_id)
emotion_df = emotion_df[["content"]]
print(emotion_df.shape[0])
emotion_df.head()

40000


,content
0,i know i was listenin to bad habit earlier an...
1,layin n bed with a headache ughhhh...waitin o...
2,funeral ceremony...gloomy friday...
3,wants to hang out with friends soon!
4,we want to trade with someone who has houston ...


In [10]:
sentiment140_df = pd.read_csv(SENTIMENT140_PATH, encoding = "ISO-8859-1", engine='python', header=None)
sentiment140_df.columns = [0, 1, 2, 3, 4, "content"]
sentiment140_df['content'] = sentiment140_df['content'].astype(str).str.lower()
sentiment140_df["content"] = sentiment140_df["content"].apply(remove_html)
sentiment140_df["content"] = sentiment140_df["content"].apply(remove_twitter_id)
sentiment140_df = sentiment140_df[["content"]]
print(sentiment140_df.shape[0])
sentiment140_df.head()

1600000


,content
0,"http://twitpic.com/2y1zl - awww, that's a bumm..."
1,is upset that he can't update his facebook by ...
2,i dived many times for the ball. managed to sa...
3,my whole body feels itchy and like its on fire
4,"no, it's not behaving at all. i'm mad. why am ..."


In [11]:
twitter_df = pd.read_csv(TWITTER_SENTIMENTS_PATH, error_bad_lines=False)
twitter_df.columns = [0, 1, 2, "content"]
twitter_df['content'] = twitter_df['content'].astype(str).str.lower()
twitter_df["content"] = twitter_df["content"].apply(remove_html)
twitter_df["content"] = twitter_df["content"].apply(remove_twitter_id)
twitter_df = twitter_df[["content"]]
print(twitter_df.shape[0])
twitter_df.head()

b'Skipping line 8836: expected 4 fields, saw 5\n'
b'Skipping line 535882: expected 4 fields, saw 7\n'


1578612


,content
0,is so sad for my apl friend.............
1,i missed the new moon trailer...
2,omg its already 7:30 :o
3,.. omgaga. im sooo im gunna cry. i've been at...
4,i think mi bf is cheating on me!!! t_t


In [12]:
full_df = pd.concat([emotion_df, sentiment140_df, twitter_df])
print(full_df.shape)
pandas_to_train_test_file(full_df, "twitter_3m")
full_df.head()

(3218612, 1)


,content
0,i know i was listenin to bad habit earlier an...
1,layin n bed with a headache ughhhh...waitin o...
2,funeral ceremony...gloomy friday...
3,wants to hang out with friends soon!
4,we want to trade with someone who has houston ...


In [0]:
# !python examples/run_language_modeling.py \
#     --output_dir=../roberta-tmp \
#     --model_type=roberta \
#     --model_name_or_path=roberta-base \
#     --do_train \
#     --train_data_file=../emotion.train.txt \
#     --do_eval \
#     --eval_data_file=../emotion.test.txt \
#     --mlm

In [0]:
!python examples/run_language_modeling.py \
    --output_dir='/content/drive/My Drive/input/models/roberta-twitter3m' \
    --model_type=roberta \
    --model_name_or_path=roberta-base \
    --do_train \
    --train_data_file=../twitter_3m.train.txt \
    --do_eval \
    --eval_data_file=../twitter_3m.test.txt \
    --save_steps=0 \
    --mlm

2020-05-02 12:22:43.455656: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
05/02/2020 12:22:46 - INFO - transformers.training_args -   PyTorch: setting up devices
05/02/2020 12:22:46 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
05/02/2020 12:22:46 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='/content/drive/My Drive/input/models/roberta-twitter3m', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluate_during_training=False, per_gpu_train_batch_size=8, per_gpu_eval_batch_size=8, gradient_accumulation_steps=1, learning_rate=5e-05, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, warmup_steps=0, logging_dir=None, logging_first_step=False, logging_steps=500, save_steps=0, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_op